# Manning liveProject: Deploy DL Model on Web & Mobile Using TensorFlow
---
## Milestone 1: Build an image classifier
## Partial Solutions Notebook
---
#### Date updated:  02-March-2021
#### Author:  Nidhin Pattaniyil & Reshama Shaikh


In [ ]:
from datetime import date
from datetime import datetime

current_date = date.today()
print("Today's date:", current_date)

In [ ]:
now1 = datetime.now()

start_time = now1.strftime("%H:%M:%S")
print("Start Time =", start_time)

In [ ]:
# run this once (each session) if `watermark` library is not loaded, then comment out
!pip install watermark tensorflow==2.3.* -q

In [ ]:
# import libraries
import requests
import glob
from io import BytesIO
import numpy as np
import os
import shutil
import pprint
import json
from pprint import pprint


In [ ]:
import IPython.display as display
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
import tensorflow as tf
import pathlib
import tensorflow_hub as hub

In [ ]:
import watermark
%load_ext watermark
#%reload_ext watermark

In [ ]:
# see version of system, python and libraries
%watermark -n -v -m -g -iv

In [ ]:
tf.__version__


# Running GPU on Colab
Ensure that GPU is running on this Colab notebook by following below steps.
1. Colab Menu: Select "Runtime"
2. "Change runtime type"
3. Select "Hardware Accelerator" = GPU
4. Save

In [ ]:
# confirm that GPU is running
tf.config.list_physical_devices('GPU')


In [ ]:
tf.test.gpu_device_name()

# Setup Project Folder

In [ ]:
!pwd

In [ ]:
!ls -lF

In [ ]:
PROJECT_NAME = "project_food_dl"

In [ ]:
# create a sub-directory for the data
# run this once and comment out
!mkdir -p {PROJECT_NAME}

In [ ]:
!ls -lF

In [ ]:
!ls -lF {PROJECT_NAME}

In [ ]:
# remove log files from models
!rm -rf {PROJECT_NAME}/artifacts


In [ ]:
!rm -rf {PROJECT_NAME}/data/food-101.tar.gz

In [ ]:
!rm -f artifacts.zip

In [ ]:
# create a sub-directory for data
!mkdir -p {PROJECT_NAME}/data

In [ ]:
!ls {PROJECT_NAME} -lF

Artifacts is common ML term used to describe the output created by the training process.

The output could be a fully trained model, a model checkpoint (for resuming training later), or simply a file created during the training process such as an image generated while training a Generative Adversarial Network (GAN).  
In the case of a Deep Learning model, the model artifacts are the trained weights stored in a binary format.


In [ ]:
# create a sub-directory for artifacts
!mkdir -p {PROJECT_NAME}/artifacts

In [ ]:
!ls {PROJECT_NAME} -lF

# Get Data

In [ ]:
! wget https://lp-prod-resources.s3-us-west-2.amazonaws.com/other/Deploying+a+Deep+Learning+Model+on+Web+and+Mobile+Applications+Using+TensorFlow/Food+101+-+Data+Subset.zip -P {PROJECT_NAME}/data


In [ ]:
# unpack the data
# run only once, then comment out 

!unzip -q {PROJECT_NAME}/data/Food+101+-+Data+Subset.zip -d {PROJECT_NAME}/data


In [ ]:
!ls {PROJECT_NAME} -lF

In [ ]:
DATA_DIR = str(PROJECT_NAME)+"/data/food-101-subset/images"
DATA_DIR = pathlib.Path(DATA_DIR)

In [ ]:
!rm -rf {DATA_DIR}/.DS_Store

In [ ]:
DATA_DIR


# Look at dataset

In [ ]:
# look at folder names
!ls -lah {DATA_DIR}/ | head 

In [ ]:
# look at first five images in first image folder
!ls {DATA_DIR}/apple_pie | head -5

In [ ]:
# find out how many total images there are in database
image_count = len(list(DATA_DIR.glob('*/*.jpg')))
image_count

In [ ]:
# find out how many different classes there are
ALL_CLASS_NAMES = sorted(np.array([item.name for item in DATA_DIR.glob('*')]))
print(len(ALL_CLASS_NAMES))

In [ ]:
ALL_CLASS_NAMES[:10]

In [ ]:
 USE_CLASS_NAMES = ALL_CLASS_NAMES

# Look at Images

In [ ]:
class1 = ALL_CLASS_NAMES[0]

In [ ]:
images = list(DATA_DIR.glob(f'{class1}/*'))

for image_path in images[:2]:
    # resize image
    im = Image.open(str(image_path))
    w, h = im.size
    print('Image Size (w, h): ', w, ",",  h)
    print (image_path)
    percent_resize = 0.5
    im = im.resize((int(w*percent_resize), int(h*percent_resize)))
    display.display(im)

# Setup for Training Model


The `ImageDataGenerator` is used to create training and validation splits.
It also has several builtin image preprocessing transformations. 

https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator

In [ ]:
BATCH_SIZE = 32
IMG_HEIGHT = 224
IMG_WIDTH = 224
STEPS_PER_EPOCH = np.ceil(image_count/BATCH_SIZE)

In [ ]:
print("Number of classes we are training: " ,len(USE_CLASS_NAMES))
print("\nList of classes")
list(USE_CLASS_NAMES)[:10]

In [ ]:
# create a data generator object with options (location of images, batch size, option to shuffle, etc)
def get_image_data_generator(preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input ):
  image_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    validation_split=0.2,
    preprocessing_function=preprocessing_function
  )



  # create a data generator object with options (location of images, batch size, option to shuffle, etc)
  image_data_gen = image_generator.flow_from_directory(
      directory=str(DATA_DIR),
      batch_size=BATCH_SIZE,
      shuffle=True,
      target_size=(IMG_HEIGHT, IMG_WIDTH),
      classes = list(USE_CLASS_NAMES)
      )

  return image_data_gen

In [ ]:
image_data_gen = get_image_data_generator (preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input)

# Save list of classes as `classes.json`

In [ ]:
image_data_gen.num_classes

In [ ]:
image_data_gen.class_indices.keys()

In [ ]:
list_of_classes = list(image_data_gen.class_indices.keys())

In [ ]:
list_of_classes

In [ ]:
with open(f"{PROJECT_NAME}/artifacts/classes.json",'w') as f:
  json.dump(list_of_classes,f)

---

# Model Architectures

### Model 1:  VGG19 (Baseline)

In [ ]:
?tf.keras.layers.Dense

In [ ]:
IMAGE_SHAPE = (IMG_HEIGHT, IMG_WIDTH)

# Use VGG19 pretrained on ImageNet
base_layers = tf.keras.applications.VGG19(weights='imagenet',include_top=False,input_shape=IMAGE_SHAPE+(3,) )

# Add new layers to be finetuned
# The last layer, is the classification layer and should match the number of classes in the dataset. The activation should be softmax 
clf = tf.keras.Sequential([
    base_layers
    , tf.keras.layers.GlobalAveragePooling2D()
    , tf.keras.layers.Dense(1024, activation='relu')
    , tf.keras.layers.Dense(image_data_gen.num_classes , name='classification', activation='softmax')
])

In [ ]:
clf.summary()

In [ ]:
# freezes the base layers
base_layers.trainable = False

In [ ]:
# notice that after freezing the base layers, the non trainable params are equal to the number of parameters in the base layer 
clf.summary()

In [ ]:
# Set the model to use Adam optimizer , cross entropy loss, and track accuracy.
# Since the dataset has multiple classes, we are using cross entropy loss.
clf.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss='categorical_crossentropy' ,
  metrics=['acc'])

#### Model results

In [ ]:
# train the model for 3 epochs
%%time
# Note the preprocessing function uses the preprocessing function for vgg19. You should replace this line for other models
image_data_gen = get_image_data_generator (preprocessing_function=tf.keras.applications.vgg19.preprocess_input)

history = clf.fit(image_data_gen
                        ,epochs=3
                        ,workers=8 
                        )


#### Save model

In [ ]:
# save the model as `h5` format
export_path = str(PROJECT_NAME)+"/artifacts/model_VGG19.h5"
export_path
clf.save(export_path, save_format='h5')


### Model 2:  ResNet50
On your own, train a model using ResNet50.  

Don't forget to use the right preprocessing function when creating the data generator.

### Model 3: MobileNetV2 (Final)
On your own, train a model using MobileNetV2.

Don't forget to use the right preprocessing function when creating the data generator.

## Fine tune model (OPTIONAL)

The model accuracy can be further improved by 
- unfreezing the early layers, use [transfer learning](https://www.tensorflow.org/guide/keras/transfer_learning)
- use [data augmentation](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator)

# Time to Train

In [ ]:
now2 = datetime.now()

end_time = now2.strftime("%H:%M:%S")
print("End Time =", end_time)

In [ ]:
diff2=((now2-now1).total_seconds() )/ (60)
print("Time to run (minutes): ", diff2)

# Get Model Size

In [ ]:
! du -h {PROJECT_NAME}/artifacts

In [ ]:
!ls {PROJECT_NAME}/artifacts/ -lah

# Compare Models

- Dataset:  Food
- Classes: 3
- Total images: 3000
- Batch size: 32

| Model  | Trainable Params| Non-trainable Params | Time (hh:mm:ss) [a]| Accuracy | Epochs | Model Size [b]
|---|---:|---:|---:|---:| ---:|---:|
| VGG19| 528,387   | 20,024,384 | 00:00:48 | 96.8% | 3 | 83M
| ResNet50   | xxx   | xxx |  xx:xx:xx | xx.x% | 3 | xxM
| MobileNetV2  | xxx  | xxx |   xx:xx:xx | xx.x% | 3 | xxM
| ResNet50_ft [c]  | xxx  | xxx | xx:xx:xx   | xx.x% | xx | xxM

NOTES:  
- [a] If wall clock time < CPU time, then you're executing a program in parallel.
- [b] model size is size of output file
- [c] ft = fine-tuned; time 2min 31s; 3min 59s; epochs 9+5


# Model Prediction

In [ ]:
model_path_vgg19 = str(PROJECT_NAME)+"/artifacts/model_VGG19.h5"
clf_final = tf.keras.models.load_model(model_path_vgg19)

In [ ]:
with open(f"{PROJECT_NAME}/artifacts/classes.json",'r') as f:
  list_of_classes = json.load(f)
  #content = f.read()

In [ ]:
list_of_classes

In [ ]:
# if you want to delete a directory from past runs
!rm -rf {PROJECT_NAME}/test_image

In [ ]:
# create a sub-directory for data
!mkdir -p {PROJECT_NAME}/test_image

In [ ]:
!wget https://natashaskitchen.com/wp-content/uploads/2019/01/Caesar-Salad-Recipe-3.jpg -P {PROJECT_NAME}/test_image -O {PROJECT_NAME}/test_image/caesar_salad.jpg

In [ ]:
!wget https://upload.wikimedia.org/wikipedia/commons/9/99/Black_square.jpg -P {PROJECT_NAME}/test_image -O {PROJECT_NAME}/test_image/black_square.jpg

In [ ]:
# 
!wget https://image.shutterstock.com/image-photo/brown-light-wooden-round-dining-260nw-588358070.jpg -P {PROJECT_NAME}/test_image  -O {PROJECT_NAME}/test_image/table.jpg


In [ ]:
img_path = f"{PROJECT_NAME}/test_image/caesar_salad.jpg"
#img_path = f"{PROJECT_NAME}/test_image/table.jpg"
#img_path = f"{PROJECT_NAME}/test_image/black_square.jpg"

img_path

In [ ]:
tf.keras.preprocessing.image.load_img(img_path, target_size = (IMG_HEIGHT, IMG_WIDTH))


In [ ]:
def load_img_predict(img_path):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size = (IMG_HEIGHT, IMG_WIDTH))
    img = tf.keras.preprocessing.image.img_to_array(img)
    img = tf.keras.applications.vgg19.preprocess_input(img)

    img = np.expand_dims(img, axis = 0)
    
    return img

In [ ]:
#classifier = classifier['MobileNetV2']
#classifier = classifier['VGG19']
clf_final.summary()

In [ ]:
def predict_image(img_path,classifier):
    img = load_img_predict(img_path)
    res = clf_final.predict(img)

    res = sorted (
        list(zip ( 
            list_of_classes
            , np.squeeze(res)
         )
        )
     , key=lambda x: x[1]   
     , reverse=True
    )
    
    return res

In [ ]:
predict_image(img_path,clf_final)[:15]

# Download Assets
download model and classes.json

In [ ]:
!zip -r {PROJECT_NAME}/artifacts/artifacts.zip {PROJECT_NAME}/artifacts/

In [ ]:
from google.colab import files
files.download(str(str(PROJECT_NAME)+"/artifacts/artifacts.zip"))